# Introduction to RAMP platform and interaction with scikit-learn

RAMP is a Kaggle-like platform. It is used to run data science challenge. Indeed, a challenge is organized around a specific problem for which the data and the evaluation are already defined. Participants will only have to focus on the development of the machine learning pipeline.

Here, we will present how the RAMP platform works. RAMP relies on a data science problem which is formulated in `problem.py`. It defines both data and evaluation. If you are interested, you can open this file. Otherwise, we will only used a couple of the function defined there.

In [ ]:
import problem

We will load the training and testing datasets available for the challenge.

In [ ]:
X_train, y_train = problem.get_train_data()

In [ ]:
X_test, y_test = problem.get_test_data()

Succently, we can check the type of features in `X` and the target that we would like to predict `y`.

In [ ]:
X_train.head()

In [ ]:
y_train[:5]

The target `y` corresponds to a number of passengers (modified using a `log` function). Associated with each target, we have an information in `X` related to the date (`DateOfDeparture`) and the airports of departure (`Departure`) and arrival (`Arrival`). Besides, we have the information regarding the mean (`WeeksToDeparture`) and standard deviation (`std_wtd`) of the time in weeks between the booking and the departure.

So we try to answer to the following information: **With some flying information between airports, can we predict the (log) flow of passengers?**

Let's make a basic scikit-learn model that could use some the data in `X` to answer this question. We will create a factory function `get_estimator()` to return the model.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor

def get_estimator():
    cat_processor = OrdinalEncoder()
    cat_columns = ["Departure", "Arrival"]

    num_processor = "passthrough"
    num_columns = ["WeeksToDeparture", "std_wtd"]

    preprocessor = make_column_transformer(
        (cat_processor, cat_columns),
        (num_processor, num_columns),
        remainder="drop",  # drop the unused columns
    )

    return make_pipeline(preprocessor, RandomForestRegressor())

In [ ]:
import sklearn
sklearn.set_config(display="diagram")
sklearn.__version__

In [ ]:
estimator = get_estimator()
estimator

At this stage, we could train and test our model using scikit-learn

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score

model = get_estimator()
cv = problem.get_cv(X_train, y_train)

scores = cross_val_score(
    model, X_train, y_train, cv=cv,
    scoring="neg_mean_squared_error",
)
rmse_scores = np.sqrt(-scores)
for cv_idx, score in enumerate(rmse_scores):
    print(f"CV Fold #{cv_idx}: {score:.3f}")
print(
    f"RMSE = {rmse_scores.mean():.3f} "
    f"+/- {rmse_scores.std():.3f}"
)

In [ ]:
y_pred = model.fit(X_train, y_train).predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

score = np.sqrt(mean_squared_error(y_test, y_pred))
score

RAMP was developed to avoid running the last cell. Instead, the idea is to store the content of cell where `get_estimator` is defined into a file. For this challenge, the name of the file is called `estimator.py`. You can check the content of `submissions/starting_kit` directory.

Let's make our first kit. Alongside of the three kits, create a folder `my_first_kit` in the folder `submissions`. The command below will create a `estimator.py` file into this directory.

In [ ]:
%%writefile submissions/my_first_kit/estimator.py

from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor

def get_estimator():
    cat_processor = OrdinalEncoder()
    cat_columns = ["Departure", "Arrival"]

    num_processor = "passthrough"
    num_columns = ["WeeksToDeparture", "std_wtd"]

    preprocessor = make_column_transformer(
        (cat_processor, cat_columns),
        (num_processor, num_columns),
        remainder="drop",  # drop the unused columns
    )

    return make_pipeline(preprocessor, RandomForestRegressor())

Once the `estimator.py` was created, we can use the `ramp-test` command to automatically execute the evaluation on our newly created estimator. 

In [ ]:
!ramp-test --submission my_first_kit

This command is the exact replica of what would happen on our server when you will submit your file `estimator.py` on https://ramp.studio.

We can now make a demo on submitting this kit on the RAMP platform.